In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# origin: petrustraße

origin = gpd.GeoDataFrame(columns = ['name', 'geometry'], crs = 4326, geometry = 'geometry')

origin.at[0, 'geometry'] = Point(6.647576, 49.760217)
origin.at[0, 'name'] = 'petrustraße'

# destination: Engnelstraße

destination = gpd.GeoDataFrame(columns = ['name', 'geometry'], crs = 4326, geometry = 'geometry')
destination.at[0, 'geometry'] = Point(6.642249, 49.761701)
destination.at[0,'name'] = 'Engelstraße'


In [3]:
origin

,name,geometry
0,petrustraße,POINT (6.64758 49.76022)


In [4]:
destination

,name,geometry
0,Engelstraße,POINT (6.64225 49.76170)


In [7]:
def osm_network(origin, destination, network = 'drive'):
    
    # creating a boundry for fetching OSM data
    combined = origin.append(destination)
    convex = combined.unary_union.convex_hull
    graph_extent = convex.buffer(0.02)

    # fetching graph
    graph = ox.graph_from_polygon(graph_extent, network_type= network)

    # get the edges / network as geodataframe
    edges = ox.graph_to_gdfs(graph, nodes = False)

    return edges

In [12]:
#osm_network(origin, destination)
# fig, ax = ox.plot_graph(osm_network(origin, destination))

osm_network(origin, destination)

C:\Users\looka\AppData\Local\Temp\ipykernel_17832\3189890786.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = origin.append(destination)


osmid  oneway lanes  \
u          v          key                                        
10602396   268656889  0     [620480498, 6073765]    True     1   
           3328610208 0                 44503497    True   NaN   
           20909583   0                113363975    True     1   
10602410   3212415597 0                  6073363   False   NaN   
           10602396   0    [348744826, 44181036]    True     3   
...                                          ...     ...   ...   
9588386667 258184403  0                  6073417   False   NaN   
           5305250272 0               1041496032    True   NaN   
9589542590 289374929  0               1041636614   False   NaN   
           9589542595 0               1041636613    True   NaN   
9589542595 9589542590 0               1041636615    True   NaN   

                                                name  \
u          v          key                              
10602396   268656889  0    [Neustraße, Germanstraße]   
           3328610208 0                 Kaiserstraße   
           20909583   0                          NaN   
10602410   3212415597 0                          NaN   
           10602396   0                 Kaiserstraße   
...                                              ...   
9588386667 258184403  0           Franz-Georg-Straße   
           5305250272 0           Franz-Georg-Straße   
9589542590 289374929  0           Auf dem Petrisberg   
           9589542595 0           Auf dem Petrisberg   
9589542595 9589542590 0           Auf dem Petrisberg   

                                                highway reversed   length  \
u          v          key                                                   
10602396   268656889  0    [living_street, residential]    False  249.956   
           3328610208 0                       secondary    False  153.907   
           20909583   0                  secondary_link    False   36.470   
10602410   3212415597 0                     residential    False   28.040   
           10602396   0                       secondary    False  219.652   
...                                                 ...      ...      ...   
9588386667 258184403  0                        tertiary     True  206.896   
           5305250272 0                        tertiary    False   44.233   
9589542590 289374929  0                     residential    False   26.152   
           9589542595 0                     residential    False   37.957   
9589542595 9589542590 0                     residential    False   37.846   

                          maxspeed  \
u          v          key            
10602396   268656889  0         50   
           3328610208 0         50   
           20909583   0         50   
10602410   3212415597 0        NaN   
           10602396   0         50   
...                            ...   
9588386667 258184403  0         30   
           5305250272 0         30   
9589542590 289374929  0         30   
           9589542595 0         30   
9589542595 9589542590 0         30   

                                                                    geometry  \
u          v          key                                                      
10602396   268656889  0    LINESTRING (6.63723 49.75009, 6.63724 49.75015...   
           3328610208 0    LINESTRING (6.63723 49.75009, 6.63711 49.75012...   
           20909583   0    LINESTRING (6.63723 49.75009, 6.63717 49.75006...   
10602410   3212415597 0    LINESTRING (6.64015 49.74952, 6.64008 49.74938...   
           10602396   0    LINESTRING (6.64015 49.74952, 6.64005 49.74955...   
...                                                                      ...   
9588386667 258184403  0    LINESTRING (6.66388 49.76727, 6.66384 49.76721...   
           5305250272 0    LINESTRING (6.66388 49.76727, 6.66395 49.76732...   
9589542590 289374929  0      LINESTRING (6.66594 49.75361, 6.66567 49.75345)   
           9589542595 0    LINESTRING (6.66594 49.75361, 6.66609 49.75366...  

In [ ]:
G = ox.graph_from_point(center_point=(6.647576, 49.760217),dist_type="network")
fig, ax = ox.plot_graph(G)
# fig, ax =  ox.plot_graph(osm_network(origin, destination))
# plt.show()

In [22]:
from pandas import MultiIndex


def shortest_path(origin, destination, network = 'drive'):

    # creating a boundry for fetching OSM data
    combined = origin.append(destination)
    convex = combined.unary_union.convex_hull
    graph_extent = convex.buffer(0.02)

    # fetching graph
    graph = ox.graph_from_polygon(graph_extent, network_type= network)


    # reproject the graph
    graph_proj = ox.project_graph(graph)

    
    # get the edges / network as geodataframe
    edges = ox.graph_to_gdfs(graph, nodes = False)

    # get CRS info UTM
    CRS = edges.crs


    # reproject all data 
    origin_proj = origin.to_crs(crs = CRS)
    destination_proj = destination.to_crs(crs = CRS)


    # routes of shortest path
    routes = gpd.GeoDataFrame() ## declaring a veriable of geodataframe

    # get nodes from the graph
    nodes = ox.graph_to_gdfs(graph_proj, edges = False)

    # iterate over origins and destinations
    for oidx, orig in origin_proj.iterrows():
        # Find closest node from the graph → point = (latitude, longitude)
        closest_origin_node = ox.nearest_nodes(G = graph_proj, Y = orig.geometry.y, X = orig.geometry.x)

        #iterate over targets
        for  tidx, target in destination_proj.iterrows():
             # Find closest node from the graph → point = (latitude, longitude)
             closest_target_node = ox.nearest_nodes(G = graph_proj, Y = target.geometry.y, X = target.geometry.x)

             # # Check if origin and target nodes are the same → if they are → skip

             if closest_origin_node == closest_target_node:
                print('same origin and destination node')

                # find the shortrest path between the points
                route = nx.shortest_path(graph_proj, source = closest_origin_node, target= closest_target_node, weight='length')

                # extract the nodes of the route
                route_nodes = nodes.loc[route]

                # create the linestring out of the route
                try:
                    path = LineString(list(route_nodes.geometry.values))
                except ValueError:
                    continue

                # append the result into the geodataframe
                routes = routes.append([[path]], ignore_index = True)
    
    # add a column name
    routes.columns = ['geometry']

    # set coordinate reference system
    routes.crs = nodes.crs

    # set geometry
    routes = routes.set_geometry('geometry')

    return routes
                 

In [ ]:
shortest_path(origin, destination)

In [16]:
# Kamppi shopping center as Origin
origin = gpd.GeoDataFrame(columns = ['name', 'geometry'], crs = 4326, geometry = 'geometry')
origin.at[0, 'geometry'] = Point(24.933260, 60.169111)
origin.at[0, 'name'] = 'Kamppi'
# Physicum
destination = gpd.GeoDataFrame(columns = ['name', 'geometry'], crs = 4326, geometry = 'geometry')
destination.at[0, 'geometry'] = Point(24.962608, 60.205301)
destination.at[0, 'name'] = 'Physicum'

In [17]:
def osm_network(origin, destination, network = 'drive'):
    
    # creating a boundary for fetching OSM data
    combined = origin.append(destination)
 
    convex = combined.unary_union.convex_hull
 
    graph_extent = convex.buffer(0.02)
    # fetching graph
    graph = ox.graph_from_polygon(graph_extent, network_type= network)
    # Get the edges/network as GeoDataFrame
    edges = ox.graph_to_gdfs(graph, nodes=False)
       
    return edges

In [18]:
osm_network(origin, destination)

C:\Users\looka\AppData\Local\Temp\ipykernel_17832\1729804832.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = origin.append(destination)


osmid  oneway  \
u           v          key                                               
25216594    1519889325 0       [23717777, 61431074, 607816332]    True   
            1372425714 0                              23856784    True   
25238874    1372425713 0                  [29977177, 30470347]    True   
            60469897   0      [73064578, 122961573, 930820886]    True   
25238944    25216594   0                  [372440330, 8135861]    True   
...                                                        ...     ...   
9744080426  33980886   0    [1093516857, 24474180, 1060577926]    True   
9749183692  9757484042 0                            1093517962    True   
9757484042  9744072416 0              [1093517963, 1060577925]    True   
9757484055  9749183692 0                            1093517961    True   
10057546868 76468537   0      [1098438089, 72728809, 72728813]   False   

                             lanes                name        highway  \
u           v          key                                              
25216594    1519889325 0         2       Porkkalankatu        primary   
            1372425714 0         2      Mechelininkatu        primary   
25238874    1372425713 0         3      Mechelininkatu        primary   
            60469897   0    [2, 1]        Itämerenkatu       tertiary   
25238944    25216594   0         2                 NaN        primary   
...                            ...                 ...            ...   
9744080426  33980886   0    [2, 3]  Hermannin rantatie        primary   
9749183692  9757484042 0         2      Kyläsaarenkatu        primary   
9757484042  9744072416 0         2      Kyläsaarenkatu        primary   
9757484055  9749183692 0         2      Kyläsaarenkatu        primary   
10057546868 76468537   0       NaN         Topparikuja  living_street   

                            maxspeed reversed   length  \
u           v          key                               
25216594    1519889325 0    [50, 40]    False  322.399   
            1372425714 0          40    False   40.885   
25238874    1372425713 0          40    False   16.601   
            60469897   0          30    False  109.773   
25238944    25216594   0          40    False   25.190   
...                              ...      ...      ...   
9744080426  33980886   0    [50, 40]    False  523.914   
9749183692  9757484042 0          30    False   16.216   
9757484042  9744072416 0          30    False   39.785   
9757484055  9749183692 0          30    False   31.494   
10057546868 76468537   0         NaN     True  119.857   

                                                                     geometry  \
u           v          key                                                      
25216594    1519889325 0    LINESTRING (24.92106 60.16479, 24.92087 60.164...   
            1372425714 0    LINESTRING (24.92106 60.16479, 24.92095 60.164...   
25238874    1372425713 0    LINESTRING (24.92103 60.16366, 24.92104 60.163...   
            60469897   0    LINESTRING (24.92103 60.16366, 24.92083 60.163...   
25238944    25216594   0    LINESTRING (24.92129 60.16463, 24.92127 60.164...   
...                                                                       ...   
9744080426  33980886   0    LINESTRING (24.96942 60.19874, 24.96927 60.198...   
9749183692  9757484042 0    LINESTRING (24.97057 60.19865, 24.97064 60.198...   
9757484042  9744072416 0    LINESTRING (24.97073 60.19876, 24.97073 60.198...   
9757484055  9749183692 0    LINESTRING (24.97006 60.19869, 24.97007 60.198...   
10057546868 76468537   0    LINESTRING (24.94165 60.20119, 24.94162 60.201...   

                              junction bridge width access  ref est_width  \
u           v          key                                                  
25216594    1519889325 0           NaN    NaN   NaN    NaN  NaN       NaN   
            1372425714 0           NaN    NaN   NaN    NaN  NaN       NaN   
25238874    1372425713 0   

In [20]:
osm_network(origin, destination, 'bike')

C:\Users\looka\AppData\Local\Temp\ipykernel_17832\1729804832.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = origin.append(destination)


osmid  oneway lanes            name  \
u           v           key                                             
25216594    1372425721  0      23717777    True     2   Porkkalankatu   
            1372425714  0      23856784    True     2  Mechelininkatu   
25238463    9443202633  0     217122749   False   NaN  Selkämerenkatu   
            8514336853  0     217122749   False   NaN  Selkämerenkatu   
            59858316    0       4229480   False   NaN             NaN   
...                                 ...     ...   ...             ...   
10128355978 10128355984 0    1106867462   False   NaN             NaN   
10128355984 10128355978 0    1106867462   False   NaN             NaN   
10129861508 3514103055  0      34203656   False   NaN             NaN   
            392270073   0      34203656   False   NaN             NaN   
            8191683816  0     880730150   False   NaN             NaN   

                                 highway maxspeed reversed   length  \
u           v           key                                           
25216594    1372425721  0        primary       40    False   10.404   
            1372425714  0        primary       40    False   40.885   
25238463    9443202633  0    residential       30    False    1.244   
            8514336853  0    residential       30     True    8.359   
            59858316    0        service       30    False   77.940   
...                                  ...      ...      ...      ...   
10128355978 10128355984 0           path      NaN    False   29.376   
10128355984 10128355978 0           path      NaN     True   29.376   
10129861508 3514103055  0           path      NaN    False    9.470   
            392270073   0           path      NaN     True   19.238   
            8191683816  0           path      NaN    False  123.394   

                                                                      geometry  \
u           v           key                                                      
25216594    1372425721  0    LINESTRING (24.92106 60.16479, 24.92087 60.16479)   
            1372425714  0    LINESTRING (24.92106 60.16479, 24.92095 60.164...   
25238463    9443202633  0    LINESTRING (24.91729 60.16004, 24.91728 60.16004)   
            8514336853  0    LINESTRING (24.91729 60.16004, 24.91738 60.15997)   
            59858316    0    LINESTRING (24.91729 60.16004, 24.91743 60.160...   
...                                                                        ...   
10128355978 10128355984 0    LINESTRING (24.95983 60.20676, 24.95985 60.206...   
10128355984 10128355978 0    LINESTRING (24.95986 60.20702, 24.95987 60.207...   
10129861508 3514103055  0    LINESTRING (24.97171 60.22088, 24.97178 60.22080)   
            392270073   0    LINESTRING (24.97171 60.22088, 24.97171 60.22105)   
            8191683816  0    LINESTRING (24.97171 60.22088, 24.97133 60.220...   

                            junction access bridge service  ref width tunnel  \
u           v           key                                                    
25216594    1372425721  0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
            1372425714  0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
25238463    9443202633  0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
            8514336853  0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
            59858316    0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
...                              ...    ...    ...     ...  ...   ...    ...   
10128355978 10128355984 0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
10128355984 10128355978 0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
10129861508 3514103055  0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
            392270073   0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   
            8191683816  0        NaN    NaN    NaN     NaN  NaN   NaN    NaN   

                            est_width area  
u      